In [10]:
import json
import sys


class Product:
    """Represents a product in the store.

    Attributes:
    - name (str): The name of the product.
    - quantity (int): The available quantity of the product.
    - purchase_price (float): The purchase price of the product.
    - selling_price (float): The selling price of the product.
    """
    def __init__(self, name, quantity, purchase_price, selling_price):
        self.name = name
        self.quantity = quantity
        self.purchase_price = purchase_price
        self.selling_price = selling_price


def help_menu():
    """Displays the available commands in the help menu."""

    print("\nCOMANDI DISPONIBILI:")
    print("1. Aggiungi prodotto - Aggiungi un nuovo prodotto")
    print("2. Lista prodotti - Mostra la lista con i prodotti presenti in negozio")
    print("3. Rimuovi prodotto - Rimuove un prodotto dalla lista")
    print("4. Registra vendita - Registra una nuova vendita")
    print("5. Profitti - Mostra i profitti lordi e netti")
    print("6. Aiuto - Mostra i comandi disponibili")
    print("7. Esci - Chiudi il programma")


def display_exit_message():
    print("\nBye Bye.")


def handle_keyboard_interrupt():
    try:
        raise KeyboardInterrupt
    except KeyboardInterrupt:
        display_exit_message()
        sys.exit()

def is_valid_product_name(name):
    """Checks if the product name is valid."""
    return any(char.isalpha() for char in name)

class VeganStore:
    """Represents a vegan store with products and sales.

    Attributes:
    - products (list): List of products in the store.
    - sales (list): List of recorded sales.
    - current_sales (list): List of sales recorded during the current interaction.
    """
    def __init__(self):
        self.products = []
        self.sales = []
        self.current_sales = []
        self.load_data()

    def add_product(self, name, quantity, purchase_price=None, selling_price=None):
        """Adds a new product to the store or updates the quantity of an existing product.

        Arguments:
        - name (str): The name of the product.
        - quantity (str): The available quantity of the product.
        - purchase_price (str, optional): The purchase price of the product (for new products).
        - selling_price (str, optional): The selling price of the product (for new products).
        """
        if not is_valid_product_name(name):
            print("Nome prodotto non valido! Il nome del prodotto non può contenere solo numeri.")
            return

        try:
            quantity = int(quantity)
            if quantity < 0:
                print("Valore non valido! La quantità non può essere negativa.")
                return

            if purchase_price is not None:
                purchase_price = float(purchase_price)
                if purchase_price < 0:
                    print("Valore non valido! Il prezzo di acquisto non può essere negativo.")
                    return

            if selling_price is not None:
                selling_price = float(selling_price)
                if selling_price < 0:
                    print("Valore non valido! Il prezzo di vendita non può essere negativo.")
                    return

        except ValueError:
            print("Valore non valido! Inserisci numeri validi per quantità e prezzi.")
            return

        for product in self.products:
            if product.name.lower() == name.lower():
                product.quantity += quantity
                self.save_data()
                print(f"Quantità di '{name}' aggiornata a {product.quantity}.")
                return

        if purchase_price is None or selling_price is None:
            print("Prodotto non trovato. Inserisci prezzo di acquisto e di vendita.")
            return

        new_product = Product(name.lower(), quantity, purchase_price, selling_price)
        self.products.append(new_product)
        self.save_data()
        print(f"AGGIUNTO: {quantity}x {name}")

    def list_products(self):
        """Displays all products in the store."""
        if not self.products:
            print("Non ci sono prodotti nel negozio.")
            return

        print("{:<15} {:<10} {:<10}".format("PRODOTTO", "QUATITA'", "PREZZO"))
        for product in self.products:
            print("{:<15} {:<10} €{:<10.2f}".format(product.name.capitalize(), product.quantity, product.selling_price))

    def remove_product(self):
        """Removes a product from the store."""
        product_name = input("Inserisci il nome del prodotto da rimuovere: ").lower()
        found = False

        for product in self.products:
            if product.name == product_name:
                found = True
                quantity_to_remove = input(f"Inserisci la quantita di prodotto da rimuovere (1 to {product.quantity}): ")
                try:
                    quantity_to_remove = int(quantity_to_remove)
                except ValueError:
                    print("Valore non valido! Inserisci un valore tra quelli indicati.")
                    return

                if 1 <= quantity_to_remove <= product.quantity:
                    product.quantity -= quantity_to_remove
                    self.save_data()
                    print(f"{quantity_to_remove}x {product_name} rimosso dallo store.")
                else:
                    print(f"Valore non valido! Inserisci un numero tra 1 e {product.quantity}.")

        if not found:
            print(f"'{product_name}' non è presente nello store.")

    def record_sale(self, product_name, sold_quantity):
        """Records a sale of a product."""
        try:
            sold_quantity = int(sold_quantity)
            if sold_quantity < 0:
                print("Valore non valido! La quantità venduta non può essere negativa.")
                return

            product_name_lower = product_name.lower()

            for product in self.products:
                if product.name.lower() == product_name_lower:
                    if product.quantity >= sold_quantity:
                        product.quantity -= sold_quantity
                        sale = {
                            'product_name': product_name_lower,
                            'sold_quantity': sold_quantity,
                            'amount': sold_quantity * product.selling_price
                        }
                        self.current_sales.append(sale)
                        self.save_data()
                        print(f"La vendita di '{product_name}' è stata registrata.")
                        return

                    else:
                        print(f"Quantità disponibile di '{product_name}' insufficiente.")
                        return

            print(f"'{product_name}' non è presente nel negozio! Vendita non registrata.")

        except ValueError:
            print("Valore non valido! Inserisci un numero valido per la quantità venduta.")

    def calculate_profits(self):
        """Calculates and displays gross and net profits."""
        
        gross_profits = sum(sale['amount'] for sale in self.current_sales)
        net_profits = gross_profits - sum(product.purchase_price * sale['sold_quantity']
                                          for sale in self.current_sales
                                          for product in self.products
                                          if product.name == sale['product_name'])
        print(f"Profitti lordi: €{gross_profits:.2f}")
        print(f"Profitti netti: €{net_profits:.2f}")
    
    def load_data(self, filename='store_data.json'):
        """Loads data from the specified JSON file."""
        
        try:
            with open(filename, 'r') as file:
                data = json.load(file)
                self.products = [Product(**product_data) for product_data in data.get('products', [])]
                self.sales = data.get('sales', [])
        except (FileNotFoundError, json.JSONDecodeError):
            pass

    def save_data(self):
        """Saves data to the store_data.json file."""
        
        data = {'products': [
            {'name': product.name, 'quantity': product.quantity, 'purchase_price': product.purchase_price,
             'selling_price': product.selling_price} for product in self.products],
                'sales': self.sales + self.current_sales}
        with open('store_data.json', 'w') as file:
            json.dump(data, file)

    def get_selling_price(self, product_name):
        """Gets the selling price of a product."""
        
        for product in self.products:
            if product.name.lower() == product_name.lower():
                return product.selling_price
        return 0

    def add_another_product(self):
        """Ask to users if they want to add another product."""
        while True:
            name = input("Nome prodotto: ")
            if name.lower() in {'aiuto', 'stop', ''}:
                break

            quantity = input("Quantità disponibile: ")
            purchase_price = input("Prezzo di acquisto: ")
            selling_price = input("Prezzo di vendità: ")
            self.add_product(name, quantity, purchase_price, selling_price)

            add_another_product = input("Vuoi aggiungere un altro prodotto? (si/no): ").lower()
            if add_another_product != 'si':
                break

    def display_current_sales(self):
        """Displays the sales recorded during the current interaction."""
        if not self.current_sales:
            return

        print("\nVendite registrate:")
        total_amount = 0

        for sale in self.current_sales:
            product_name = sale['product_name']
            sold_quantity = sale['sold_quantity']
            selling_price = self.get_selling_price(product_name)
            amount = sale['amount']
            total_amount += amount

            print(f"x{sold_quantity} {product_name.capitalize()}: €{selling_price:.2f}")

        print(f"Totale: €{total_amount:.2f}\n")


def register_another_sell(store):
    """Ask users if they want to register another sell."""
    while True:
        product_name = input("Nome del prodotto venduto: ")
        if product_name.lower() in {'aiuto', 'stop', ''}:
            break

        sold_quantity = input("Quantità venduta: ")
        store.record_sale(product_name, sold_quantity)

        register_another_sale = input("Vuoi registrare un'altra vendit? (si/no): ").lower()
        if register_another_sale != 'si':
            break


store = VeganStore()

while True:
    try:
        command = input("\nInserisci un comando: ")

        if command.lower() in {'1', 'aggiungi prodotto'}:
            store.add_another_product()

        elif command.lower() in {'2', 'lista prodotti'}:
            store.list_products()

        elif command.lower() in {'3', 'rimuovi prodotto'}:
            store.remove_product()

        elif command.lower() in {'4', 'registra vendita'}:
            register_another_sell(store)
            store.display_current_sales()

        elif command.lower() in {'5', 'profitti'}:
            store.calculate_profits()

        elif command.lower() in {'6', 'aiuto', ''}:
            help_menu()

        elif command.lower() in {'7', 'Esci'}:
            display_exit_message()
            break

        else:
            print("Comando non valido. Inserisci 'aiuto' per vedere i comandi disponibili e una breve descrizione.")
    except KeyboardInterrupt:
        handle_keyboard_interrupt()



Inserisci un comando: 1
Nome prodotto: 301
Quantità disponibile: 30
Prezzo di acquisto: 1.99
Prezzo di vendità: 2.99
Nome prodotto non valido! Il nome del prodotto non può contenere numeri.
Vuoi aggiungere un altro prodotto? (si/no): no

Inserisci un comando: 7

Bye Bye.
